<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/LogisticRegStandardExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 54.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4a83fcdb1adf5f03fb4b1bf8a18c4031bb3e877e3bc8f553461d36301e6f6a0a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

my_data = spark.read.format('libsvm').load("sample_libsvm_data.txt")

In [4]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
my_log_reg_model = LogisticRegression()
fitted_logreg = my_log_reg_model.fit(my_data)


In [6]:
log_summary = fitted_logreg.summary
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [9]:
log_summary.predictions.select('label', 'prediction').show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [10]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [11]:
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)

In [12]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [13]:
prediction_and_labels.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[24.7950468813347...|[0.99999999998295...|       0.0|
|  0.0|(692,[122,123,148...|[23.9815703589148...|[0.99999999996154...|       0.0|
|  0.0|(692,[123,124,125...|[36.6284422851403...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[34.1630045639150...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[36.0180627886247...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[39.3430625282505...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[13.2227512043701...|[0.99999819103014...|       0.0|
|  0.0|(692,[126,127,128...|[26.2580896160959...|[0.99999999999605...|       0.0|
|  0.0|(692,[127,128,129...|[22.9616774457710...|[0.99999999989337...|       0.0|
|  0.0|(692,[152

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

my_eval = BinaryClassificationEvaluator()
my_roc =  my_eval.evaluate(prediction_and_labels.predictions)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [16]:
my_roc

1.0